In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
load_dotenv()  # automatically reads .env



dtype = {
    "RatecodeID": "Int32",
    "payment_type": "Int32",
    "trip_type": "Int32"
}

In [88]:
trip_data = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-11.parquet")
trip_data = trip_data.astype(dtype)
zones = pd.read_csv("https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv")


In [89]:
trip_data.sample(5)

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee
39045,2,2025-11-29 10:38:17,2025-11-29 10:55:38,N,1,236,138,1.0,8.32,33.80,...,0.5,9.45,6.94,NaN,1.0,56.69,1,1,0.0,0.0
2344,1,2025-11-02 20:36:36,2025-11-02 20:38:59,N,1,33,66,1.0,0.40,4.40,...,1.5,0.00,0.00,NaN,1.0,6.90,2,1,0.0,0.0
43652,6,2025-11-13 07:12:52,2025-11-13 08:37:08,NaN,<NA>,39,234,NaN,10.16,1.45,...,0.5,0.00,0.00,NaN,0.3,46.24,<NA>,<NA>,NaN,0.0
24887,2,2025-11-18 18:30:35,2025-11-18 18:34:11,N,1,43,151,1.0,0.88,6.50,...,0.5,0.74,0.00,NaN,1.0,11.24,1,1,0.0,0.0
9261,2,2025-11-07 17:09:41,2025-11-07 17:20:25,N,1,41,24,1.0,1.19,11.40,...,0.5,0.00,0.00,NaN,1.0,15.40,2,1,0.0,0.0


In [90]:
zones.sample(5)

,LocationID,Borough,Zone,service_zone
141,142,Manhattan,Lincoln Square East,Yellow Zone
5,6,Staten Island,Arrochar/Fort Wadsworth,Boro Zone
206,207,Queens,Saint Michaels Cemetery/Woodside,Boro Zone
88,89,Brooklyn,Flatbush/Ditmas Park,Boro Zone
72,73,Queens,East Flushing,Boro Zone


In [91]:
print(trip_data.shape)
print(zones.shape)


(46912, 21)
(265, 4)


In [93]:
# creating connection to my to my db 
user = os.environ["user"]
password = os.environ["password"]
host = os.environ["host"]
port = os.environ["port"]
db = os.environ["db"]

engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{db}')

In [94]:
# Just to have a look at the expected schema before creating the table

print(pd.io.sql.get_schema(trip_data, name='green_taxi_data', con=engine))
print(pd.io.sql.get_schema(zones, name='green_taxi_data', con=engine))


CREATE TABLE green_taxi_data (
	"VendorID" INTEGER, 
	lpep_pickup_datetime TIMESTAMP WITHOUT TIME ZONE, 
	lpep_dropoff_datetime TIMESTAMP WITHOUT TIME ZONE, 
	store_and_fwd_flag TEXT, 
	"RatecodeID" INTEGER, 
	"PULocationID" INTEGER, 
	"DOLocationID" INTEGER, 
	passenger_count FLOAT(53), 
	trip_distance FLOAT(53), 
	fare_amount FLOAT(53), 
	extra FLOAT(53), 
	mta_tax FLOAT(53), 
	tip_amount FLOAT(53), 
	tolls_amount FLOAT(53), 
	ehail_fee FLOAT(53), 
	improvement_surcharge FLOAT(53), 
	total_amount FLOAT(53), 
	payment_type INTEGER, 
	trip_type INTEGER, 
	congestion_surcharge FLOAT(53), 
	cbd_congestion_fee FLOAT(53)
)



CREATE TABLE green_taxi_data (
	"LocationID" BIGINT, 
	"Borough" TEXT, 
	"Zone" TEXT, 
	service_zone TEXT
)




In [95]:
# Creating a Table for both dataframes

trip_data.head(n=0).to_sql(name='green_taxi_trips', con=engine, if_exists='replace',index=False)
print("'green_taxi_trips' Table created")

zones.head(n=0).to_sql(name='green_taxi_zones', con=engine, if_exists='replace',index=False)
print("'green_taxi_zones' Table created")

'green_taxi_trips' Table created
'green_taxi_zones' Table created


In [ ]:
# Inserting the data into the existing tables.
# Since the dataset is not large I did not bother loading it in chunks

trip_data.to_sql(
    name="green_taxi_trips",
    con=engine,
    if_exists="append",
    index=False
)
print(f"Inserted {len(trip_data)} rows")

zones.to_sql(
    name="green_taxi_zones",
    con=engine,
    if_exists="append",
    index=False
)
print(f"Inserted {len(zones)} rows")

Inserted 46912 rows
Inserted 265 rows
